In [2]:
import numpy as np
from Source.utils import load_mosad_dataset
from benchmark.metrics import f_measure, covering
from Source.visualizer import plot_profile_with_ts
from Source.competitors.clasp.segmentation import clasp_segmentation
from Source.competitors.floss import floss as fl
from Source.competitors.bocd import bocd
from Source.competitors.espresso import espresso
from Source.competitors.binseg import binseg
from Source.competitors.window import window
import time


from __future__ import division

import os
import scipy.sparse
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns; sns.reset_defaults()

from scipy.io import loadmat
from sklearn.datasets import make_sparse_spd_matrix
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.utils.extmath import squared_norm
from sklearn.covariance import empirical_covariance
from importlib import reload
from regain import model_selection; reload(model_selection)
import regain
from regain.covariance import latent_time_graphical_lasso_; reload(latent_time_graphical_lasso_)
from regain.utils import error_norm

In [3]:
data = np.load('C:/Users/pc/OneDrive/Bureau/myenv/Scripts/mobile_sensing_human_activity_data_set_main/datasets/data.npz')

In [4]:
data.files

['Routine1_Subject1_X-Acc',
 'Routine1_Subject1_X-Gyro',
 'Routine1_Subject1_X-Mag',
 'Routine1_Subject1_Y-Acc',
 'Routine1_Subject1_Y-Gyro',
 'Routine1_Subject1_Y-Mag',
 'Routine1_Subject1_Z-Acc',
 'Routine1_Subject1_Z-Gyro',
 'Routine1_Subject1_Z-Mag',
 'Routine1_Subject2_X-Acc',
 'Routine1_Subject2_X-Gyro',
 'Routine1_Subject2_X-Mag',
 'Routine1_Subject2_Y-Acc',
 'Routine1_Subject2_Y-Gyro',
 'Routine1_Subject2_Y-Mag',
 'Routine1_Subject2_Z-Acc',
 'Routine1_Subject2_Z-Gyro',
 'Routine1_Subject2_Z-Mag',
 'Routine1_Subject3_X-Acc',
 'Routine1_Subject3_X-Gyro',
 'Routine1_Subject3_X-Mag',
 'Routine1_Subject3_Y-Acc',
 'Routine1_Subject3_Y-Gyro',
 'Routine1_Subject3_Y-Mag',
 'Routine1_Subject3_Z-Acc',
 'Routine1_Subject3_Z-Gyro',
 'Routine1_Subject3_Z-Mag',
 'Routine1_Subject4_X-Acc',
 'Routine1_Subject4_X-Gyro',
 'Routine1_Subject4_X-Mag',
 'Routine1_Subject4_Y-Acc',
 'Routine1_Subject4_Y-Gyro',
 'Routine1_Subject4_Y-Mag',
 'Routine1_Subject4_Z-Acc',
 'Routine1_Subject4_Z-Gyro',
 'Routin

In [5]:
np.set_printoptions(threshold=np.inf)

In [6]:
data['Routine1_Subject3_Y-Mag']

array([ 2.6714000e+01,  2.5423000e+01,  2.4067000e+01,  2.2586000e+01,
        2.1071000e+01,  1.9675000e+01,  1.8582000e+01,  1.7826000e+01,
        1.7460000e+01,  1.7258000e+01,  1.7134000e+01,  1.6907000e+01,
        1.6631000e+01,  1.6389000e+01,  1.6385000e+01,  1.6539000e+01,
        1.6695000e+01,  1.6663000e+01,  1.6446000e+01,  1.6160000e+01,
        1.6019000e+01,  1.6229000e+01,  1.6965000e+01,  1.8046000e+01,
        1.8915000e+01,  1.9575000e+01,  2.0081000e+01,  2.0714000e+01,
        2.1351000e+01,  2.1818000e+01,  2.2147000e+01,  2.1917000e+01,
        2.1290000e+01,  2.0861000e+01,  2.0638000e+01,  2.0253000e+01,
        1.9556000e+01,  1.9087000e+01,  1.9277000e+01,  2.0020000e+01,
        2.1021000e+01,  2.1934000e+01,  2.2588000e+01,  2.3189000e+01,
        2.3539000e+01,  2.3882000e+01,  2.4076000e+01,  2.3981000e+01,
        2.3576000e+01,  2.2896000e+01,  2.1982000e+01,  2.0846000e+01,
        1.9634000e+01,  1.8514000e+01,  1.7781000e+01,  1.7368000e+01,
      

In [8]:
np.random.seed(1379)

In [9]:
selection = 100
df = load_mosad_dataset().iloc[selection:selection+1,:]

In [10]:
df

,dataset,routine,subject,sensor,sample_rate,change_points,activities,time_series
100,Routine3_Subject3_X-Gyro,3,3,X-Gyro,50,"[6118, 9175, 15242, 16841, 18067]","[slow walk, run, walk, rope jump, squat, jumpi...","[0.1242, 0.1004, 0.064829, 0.041278, 0.082064,..."


In [9]:
df['time_series']

100    [0.1242, 0.1004, 0.064829, 0.041278, 0.082064,...
Name: time_series, dtype: object

In [11]:
for _, (dataset, routine, subject, sensor, sample_rate, change_points, activities, time_series) in df.iterrows():
    runtime = time.process_time()
    profile, window_size, found_cps, found_scores = clasp_segmentation(time_series, n_change_points=len(change_points), offset=int(250 / time_series.shape[0]))
    runtime = time.process_time() - runtime

    f1_score = f_measure({0: change_points}, found_cps, margin=int(time_series.shape[0] * .01))
    covering_score = covering({0: change_points}, found_cps, time_series.shape[0])

    plot_profile_with_ts(dataset, time_series, profile, change_points, found_cps, show=False) # , save_path="../simpleTest.pdf")
    print(f"{dataset}: True Change Points: {change_points}, Found Change Points: {found_cps}, F1-Score: {np.round(f1_score, 3)}, Covering-Score: {np.round(covering_score, 3)} Runtime: {np.round(runtime, 3)}")

c:\users\pc\appdata\local\programs\python\python39\lib\site-packages\Source\competitors\clasp\segmentation.py:21: UserWarning: This code is only for reproducibility. If you want to use ClaSP in your scientific publication or application, please use the updated and maintained claspy Python package: https://github.com/ermshaua/claspy
  warnings.warn(


Routine3_Subject3_X-Gyro: True Change Points: [ 6118  9175 15242 16841 18067], Found Change Points: [18088  9221  6153 15131 16765], F1-Score: 1.0, Covering-Score: 0.972 Runtime: 211.953


In [12]:
for _, (dataset, routine, subject, sensor, sample_rate, change_points, activities, time_series) in df.iterrows():
        runtime = time.process_time()
        profile, found_cps = fl(time_series, sample_rate*20, sample_rate, len(change_points), return_cac=True)
        runtime = time.process_time() - runtime

        f1_score = f_measure({0: change_points}, found_cps, margin=int(time_series.shape[0] * .01))
        covering_score = covering({0: change_points}, found_cps, time_series.shape[0])

        plot_profile_with_ts(dataset, time_series, profile, change_points, found_cps, show=False) #, save_path="../simpleTest.pdf")
        print(f"{dataset}: True Change Points: {change_points}, Found Change Points: {found_cps}, F1-Score: {np.round(f1_score, 3)}, Covering-Score: {np.round(covering_score, 3)} Runtime: {np.round(runtime, 3)}")

Routine3_Subject3_X-Gyro: True Change Points: [ 6118  9175 15242 16841 18067], Found Change Points: [16797  9115 15413 15138  6296], F1-Score: 0.833, Covering-Score: 0.889 Runtime: 56.5


In [12]:
for _, (dataset, routine, subject, sensor, sample_rate, change_points, activities, time_series) in df.iterrows():
        runtime = time.process_time()
        profile, found_cps = bocd(time_series, len(change_points), return_profile=True)
        runtime = time.process_time() - runtime

        f1_score = f_measure({0: change_points}, found_cps, margin=int(time_series.shape[0] * .01))
        covering_score = covering({0: change_points}, found_cps, time_series.shape[0])

        plot_profile_with_ts(dataset, time_series, profile, change_points, found_cps, show=False) # , save_path="../simpleTest.pdf")
        print(f"{dataset}: True Change Points: {change_points}, Found Change Points: {found_cps}, F1-Score: {np.round(f1_score, 3)}, Covering-Score: {np.round(covering_score, 3)} Runtime: {np.round(runtime, 3)}")

Routine3_Subject3_X-Gyro: True Change Points: [ 6118  9175 15242 16841 18067], Found Change Points: [  249  9967 10995 10183 14147], F1-Score: 0.167, Covering-Score: 0.454 Runtime: 72.688


In [13]:
for _, (dataset, routine, subject, sensor, sample_rate, change_points, activities, time_series) in df.iterrows():
        runtime = time.process_time()
        found_cps = window(time_series, 5*sample_rate, n_cps=len(change_points), offset=int(250 / time_series.shape[0]))
        runtime = time.process_time() - runtime

        f1_score = f_measure({0: change_points}, found_cps, margin=int(time_series.shape[0] * .01))
        covering_score = covering({0: change_points}, found_cps, time_series.shape[0])

        plot_profile_with_ts(dataset, time_series, profile, change_points, found_cps, show=False) # , save_path="../simpleTest.pdf")
        print(f"{dataset}: True Change Points: {change_points}, Found Change Points: {found_cps}, F1-Score: {np.round(f1_score, 3)}, Covering-Score: {np.round(covering_score, 3)} Runtime: {np.round(runtime, 3)}")

Routine3_Subject3_X-Gyro: True Change Points: [ 6118  9175 15242 16841 18067], Found Change Points: [ 6150  7670  8660  9255 15275], F1-Score: 0.667, Covering-Score: 0.766 Runtime: 65.375
